Testing ProtoNN and Bonsai models with dataset for this project

Code taken from: https://github.com/microsoft/EdgeML

minor model testing of LSTM model at the very end

# Adding Dataset

In [0]:
# data analysis packages
import pandas as pd
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/My Drive/Dataset/EdgeML/tf")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


# Including the relevant library

In [0]:
#!git clone https://github.com/microsoft/EdgeML.git

In [0]:

#!pip install -r EdgeML/tf/requirements-cpu.txt
#!pip install -e EdgeML/tf


In [0]:
cd EdgeML/tf

[Errno 2] No such file or directory: 'EdgeML/tf'
/content/drive/My Drive/Dataset/EdgeML/tf


In [0]:
ls

bot_iot_dataset/  examples/         requirements-cpu.txt  usps10/
docs/             helpermethods.py  requirements-gpu.txt
edgeml/           __pycache__/      sampled_dataset/
edgeml.egg-info/  README.md         setup.py


# ProtoNN

## Testing parts of old dataset

In [0]:
DATA_DIR = './usps10'
orig_train, orig_test = np.load(DATA_DIR + '/train.npy'), np.load(DATA_DIR + '/test.npy')

In [0]:

print(orig_train.shape)
print(orig_test.shape)

print(train.shape)
print(test.shape)

print(orig_train)
print(train)

#np.ptp(train,axis=1)

In [0]:
train = np.load('s_train_combined.npy')

test = np.load('s_test_combined.npy')

## Opening new dataset

In [0]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.
from __future__ import print_function
import sys
import os
import numpy as np
import tensorflow as tf

from edgeml.trainer.protoNNTrainer import ProtoNNTrainer
from edgeml.graph.protoNN import ProtoNN
import edgeml.utils as utils
import helpermethods as helper

In [0]:
def to_onehot(y, numClasses, minlabel = None):
    '''
    If the y labelling does not contain the minimum label info, use min-label to
    provide this value.
    '''
    lab = y.astype('uint8')
    if minlabel is None:
        minlabel = np.min(lab)
    minlabel = int(minlabel)
    lab = np.array(lab) - minlabel
    lab_ = np.zeros((y.shape[0], numClasses))
    lab_[np.arange(y.shape[0]), lab] = 1
    return lab_

In [0]:
# Load data
#DATA_DIR = './usps10'
DATA_DIR = "bot_iot_dataset/" #BotIoT dataset
OUT_DIR  = "bot_iot_dataset/ProtoNN_output"
#DATA_DIR = "sampled_dataset/" #N_BaIoT dataset

#train, test = np.load(DATA_DIR + '/train.npy'), np.load(DATA_DIR + '/test.npy') #three class version
#train, test = np.load(DATA_DIR + '/train_all_classes.npy'), np.load(DATA_DIR + '/test_all_classes.npy')
#train, test = np.load(DATA_DIR + '//train_15w_11c.npy'), np.load(DATA_DIR + '//test_15w_11c.npy')
train, test = np.load(DATA_DIR + '//bot_iot_train_10.npy'), np.load(DATA_DIR + '//bot_iot_test_10.npy')

x_train, y_train = train[:, 1:], train[:, 0]
x_test, y_test = test[:, 1:], test[:, 0]

numClasses = max(y_train) - min(y_train) + 1
numClasses = max(numClasses, max(y_test) - min(y_test) + 1)
numClasses = int(numClasses)

#y_train = helper.to_onehot(y_train, numClasses)
#y_test = helper.to_onehot(y_test, numClasses)

y_train = to_onehot(y_train, numClasses)
y_test = to_onehot(y_test, numClasses)

dataDimension = x_train.shape[1]
numClasses = y_train.shape[1]


In [0]:
def preprocessData(train, test):
    '''
    Loads data from the dataDir and does some initial preprocessing
    steps. Data is assumed to be contained in two files,
    train.npy and test.npy. Each containing a 2D numpy array of dimension
    [numberOfExamples, numberOfFeatures + 1]. The first column of each
    matrix is assumed to contain label information.
    For an N-Class problem, we assume the labels are integers from 0 through
    N-1.
    '''
    dataDimension = int(train.shape[1]) - 1
    x_train = train[:, 1:dataDimension + 1]
    y_train_ = train[:, 0]
    x_test = test[:, 1:dataDimension + 1]
    y_test_ = test[:, 0]

    numClasses = max(y_train_) - min(y_train_) + 1
    numClasses = max(numClasses, max(y_test_) - min(y_test_) + 1)
    numClasses = int(numClasses)

    # mean-var
    mean = np.mean(x_train, 0)
    std = np.std(x_train, 0)
    std[std[:] < 0.000001] = 1
    x_train = (x_train - mean) / std
    x_test = (x_test - mean) / std

    # one hot y-train
    lab = y_train_.astype('uint8')
    lab = np.array(lab) - min(lab)
    lab_ = np.zeros((x_train.shape[0], numClasses))
    lab_[np.arange(x_train.shape[0]), lab] = 1
    y_train = lab_

    # one hot y-test
    lab = y_test_.astype('uint8')
    lab = np.array(lab) - min(lab)
    lab_ = np.zeros((x_test.shape[0], numClasses))
    lab_[np.arange(x_test.shape[0]), lab] = 1
    y_test = lab_

    return dataDimension, numClasses, x_train, y_train, x_test, y_test

In [0]:
def getGamma(gammaInit, projectionDim, dataDim, numPrototypes, x_train):
    if gammaInit is None:
        print("Using median heuristic to estimate gamma.")
        gamma, W, B = utils.medianHeuristic(x_train, projectionDim,
                                            numPrototypes)
        print("Gamma estimate is: %f" % gamma)
        return W, B, gamma
    return None, None, gammaInit

In [0]:
def getModelSize(matrixList, sparcityList, expected=True, bytesPerVar=4):
    '''
    expected: Expected size according to the parameters set. The number of
        zeros could actually be more than that is required to satisfy the
        sparsity constraint.
    '''
    nnzList, sizeList, isSparseList = [], [], []
    hasSparse = False
    for i in range(len(matrixList)):
        A, s = matrixList[i], sparcityList[i]
        assert A.ndim == 2
        assert s >= 0
        assert s <= 1
        nnz, size, sparse = utils.countnnZ(A, s, bytesPerVar=bytesPerVar)
        nnzList.append(nnz)
        sizeList.append(size)
        hasSparse = (hasSparse or sparse)

    totalnnZ = np.sum(nnzList)
    totalSize = np.sum(sizeList)
    if expected:
        return totalnnZ, totalSize, hasSparse
    numNonZero = 0
    totalSize = 0
    hasSparse = False
    for i in range(len(matrixList)):
        A, s = matrixList[i], sparcityList[i]
        numNonZero_ = np.count_nonzero(A)
        numNonZero += numNonZero_
        hasSparse = (hasSparse or (s < 0.5))
        if s <= 0.5:
            totalSize += numNonZero_ * 2 * bytesPerVar
        else:
            totalSize += A.size * bytesPerVar
    return numNonZero, totalSize, hasSparse

In [0]:
_, _, x_train, y_train, x_test, y_test = preprocessData(train, test)

y_ = np.expand_dims(np.argmax(y_train, axis=1), axis=1)
train = np.concatenate([y_, x_train], axis=1)

y_ = np.expand_dims(np.argmax(y_test, axis=1), axis=1)
test = np.concatenate([y_, x_test], axis=1)

In [0]:

PROJECTION_DIM = 60
NUM_PROTOTYPES = 60
REG_W = 0.000005
REG_B = 0.0
REG_Z = 0.00005
SPAR_W = 0.8
SPAR_B = 1.0
SPAR_Z = 1.0
LEARNING_RATE = 0.05
#NUM_EPOCHS = 200
NUM_EPOCHS = 20
#BATCH_SIZE = 32
BATCH_SIZE = 100
GAMMA = 0.0015



In [0]:
#W, B, gamma = helper.getGamma(GAMMA, PROJECTION_DIM, dataDimension, NUM_PROTOTYPES, x_train)

W, B, gamma = getGamma(GAMMA, PROJECTION_DIM, dataDimension,
                       NUM_PROTOTYPES, x_train)

In [0]:
# Setup input and train protoNN
X = tf.placeholder(tf.float32, [None, dataDimension], name='X')
Y = tf.placeholder(tf.float32, [None, numClasses], name='Y')
protoNN = ProtoNN(dataDimension, PROJECTION_DIM,
                  NUM_PROTOTYPES, numClasses,
                  gamma, W=W, B=B)
trainer = ProtoNNTrainer(protoNN, REG_W, REG_B, REG_Z,
                         SPAR_W, SPAR_B, SPAR_Z,
                         LEARNING_RATE, X, Y, lossType='xentropy')
sess = tf.Session()
trainer.train(BATCH_SIZE, NUM_EPOCHS, sess, x_train, x_test, y_train, y_test,
              printStep=600, valStep=10)

W0717 08:54:21.328780 139790765213568 deprecation_wrapper.py:119] From /content/drive/My Drive/Dataset/EdgeML/tf/edgeml/trainer/protoNNTrainer.py:109: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0717 08:54:21.457077 139790765213568 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0717 08:54:21.534565 139790765213568 deprecation_wrapper.py:119] From /content/drive/My Drive/Dataset/EdgeML/tf/edgeml/trainer/protoNNTrainer.py:115: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 08:54:21.538416 139790765213568 deprecation_wrapper.py:119] From /content/drive/My Drive/Dataset/EdgeML/tf/edgeml/trainer/protoNNTrainer.

Epoch:   0 Batch:   0 Loss: 4.89323 Accuracy: 0.33000
Epoch:   1 Batch:   0 Loss: 0.72883 Accuracy: 0.70000
Epoch:   2 Batch:   0 Loss: 0.51062 Accuracy: 0.85000
Epoch:   3 Batch:   0 Loss: 0.43267 Accuracy: 0.84000
Epoch:   4 Batch:   0 Loss: 0.39050 Accuracy: 0.84000
Epoch:   5 Batch:   0 Loss: 0.36783 Accuracy: 0.87000
Epoch:   6 Batch:   0 Loss: 0.35228 Accuracy: 0.88000
Epoch:   7 Batch:   0 Loss: 0.34193 Accuracy: 0.90000
Epoch:   8 Batch:   0 Loss: 0.33493 Accuracy: 0.90000
Epoch:   9 Batch:   0 Loss: 0.32985 Accuracy: 0.90000
Test Loss: 0.37845 Accuracy: 0.86120
Epoch:  10 Batch:   0 Loss: 0.32600 Accuracy: 0.90000
Epoch:  11 Batch:   0 Loss: 0.32292 Accuracy: 0.91000
Epoch:  12 Batch:   0 Loss: 0.32029 Accuracy: 0.91000
Epoch:  13 Batch:   0 Loss: 0.31795 Accuracy: 0.91000
Epoch:  14 Batch:   0 Loss: 0.31574 Accuracy: 0.91000
Epoch:  15 Batch:   0 Loss: 0.31376 Accuracy: 0.90000
Epoch:  16 Batch:   0 Loss: 0.31201 Accuracy: 0.89000
Epoch:  17 Batch:   0 Loss: 0.31054 Accuracy:

In [0]:
acc = sess.run(protoNN.accuracy, feed_dict={X: x_test, Y: y_test})
# W, B, Z are tensorflow graph nodes
W, B, Z, _ = protoNN.getModelMatrices()
matrixList = sess.run([W, B, Z])
sparcityList = [SPAR_W, SPAR_B, SPAR_Z]
#nnz, size, sparse = helper.getModelSize(matrixList, sparcityList)
nnz, size, sparse = getModelSize(matrixList, sparcityList)
print("Final test accuracy", acc)
print("Model size constraint (Bytes): ", size)
print("Number of non-zeros: ", nnz)
#nnz, size, sparse = helper.getModelSize(matrixList, sparcityList, expected=False)
nnz, size, sparse = getModelSize(matrixList, sparcityList, expected=False)
print("Actual model size: ", size)
print("Actual non-zeros: ", nnz)

Final test accuracy 0.8678842
Model size constraint (Bytes):  18960
Number of non-zeros:  4740
Actual model size:  18960
Actual non-zeros:  4632


In [0]:
print("Saving model matrices to: ", OUT_DIR)
np.save(OUT_DIR + '/W.npy', matrixList[0])
np.save(OUT_DIR + '/B.npy', matrixList[1])
np.save(OUT_DIR + '/Z.npy', matrixList[2])
np.save(OUT_DIR + '/gamma.npy', gamma)

Saving model matrices to:  bot_iot_dataset/ProtoNN_output


# Bonsai

In [0]:
#cp helpermethods.py ../..

In [0]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.

import helpermethods
import tensorflow as tf
import numpy as np
import sys
import os

#Provide the GPU number to be used
#os.environ['CUDA_VISIBLE_DEVICES'] =''

#Bonsai imports
from edgeml.trainer.bonsaiTrainer import BonsaiTrainer
from edgeml.graph.bonsai import Bonsai

# Fixing seeds for reproducibility
tf.set_random_seed(42)
np.random.seed(42)

In [0]:
def preProcessData(dataDir, isRegression=False):
    '''
    Function to pre-process input data
    Expects a .npy file of form [lbl feats] for each datapoint
    Outputs a train and test set datapoints appended with 1 for Bias induction
    dataDimension, numClasses are inferred directly
    '''
    
    '''
    #eleven class version
    train = np.load(dataDir + '/train_all_classes.npy')
    test = np.load(dataDir + '/test_all_classes.npy')
    '''
    '''
    train = np.load(dataDir + '/train_100w_11c.npy')
    test = np.load(dataDir + '/test_100w_11c.npy')
    '''
    '''
    train = np.load(dataDir + '/train.npy')
    test = np.load(dataDir + '/test.npy')
    '''
    train = np.load(dataDir + '/bot_iot_train_10.npy')
    test = np.load(dataDir + '/bot_iot_test_10.npy')
    
    dataDimension = int(train.shape[1]) - 1

    Xtrain = train[:, 1:dataDimension + 1]
    Ytrain_ = train[:, 0]

    Xtest = test[:, 1:dataDimension + 1]
    Ytest_ = test[:, 0]

    # Mean Var Normalisation
    mean = np.mean(Xtrain, 0)
    std = np.std(Xtrain, 0)
    std[std[:] < 0.000001] = 1
    Xtrain = (Xtrain - mean) / std
    Xtest = (Xtest - mean) / std
    # End Mean Var normalisation

    # Classification.
    if (isRegression == False):
        numClasses = max(Ytrain_) - min(Ytrain_) + 1
        numClasses = int(max(numClasses, max(Ytest_) - min(Ytest_) + 1))

        lab = Ytrain_.astype('uint8')
        lab = np.array(lab) - min(lab)

        lab_ = np.zeros((Xtrain.shape[0], numClasses))
        lab_[np.arange(Xtrain.shape[0]), lab] = 1
        if (numClasses == 2):
            Ytrain = np.reshape(lab, [-1, 1])
        else:
            Ytrain = lab_

        lab = Ytest_.astype('uint8')
        lab = np.array(lab) - min(lab)

        lab_ = np.zeros((Xtest.shape[0], numClasses))
        lab_[np.arange(Xtest.shape[0]), lab] = 1
        if (numClasses == 2):
            Ytest = np.reshape(lab, [-1, 1])
        else:
            Ytest = lab_

    elif (isRegression == True):
        # The number of classes is always 1, for regression.
        numClasses = 1
        Ytrain = Ytrain_
        Ytest = Ytest_

    trainBias = np.ones([Xtrain.shape[0], 1])
    Xtrain = np.append(Xtrain, trainBias, axis=1)
    testBias = np.ones([Xtest.shape[0], 1])
    Xtest = np.append(Xtest, testBias, axis=1)

    mean = np.append(mean, np.array([0]))
    std = np.append(std, np.array([1]))

    if (isRegression == False):
        return dataDimension + 1, numClasses, Xtrain, Ytrain, Xtest, Ytest, mean, std
    elif (isRegression == True):
        return dataDimension + 1, numClasses, Xtrain, Ytrain.reshape((-1, 1)), Xtest, Ytest.reshape((-1, 1)), mean, std

In [0]:
#Loading and Pre-processing dataset for Bonsai

#dataDir = "sampled_dataset/"
dataDir = "bot_iot_dataset/"
(dataDimension, numClasses, Xtrain, Ytrain, Xtest, Ytest, mean, std) = preProcessData(dataDir, isRegression=False)
print("Feature Dimension: ", dataDimension)
print("Num classes: ", numClasses)

Feature Dimension:  10
Num classes:  10


In [0]:
sigma = 1.0 #Sigmoid parameter for tanh
depth = 3 #Depth of Bonsai Tree
projectionDimension = 28 #Lower Dimensional space for Bonsai to work on

#Regularizers for Bonsai Parameters
regZ = 0.0001
regW = 0.001
regV = 0.001
regT = 0.001

#totalEpochs = 100
totalEpochs = 20

learningRate = 0.01

outFile = None

#Sparsity for Bonsai Parameters. x => 100*x % are non-zeros
sparZ = 0.2
sparW = 0.3
sparV = 0.3
sparT = 0.62

batchSize = np.maximum(100, int(np.ceil(np.sqrt(Ytrain.shape[0]))))

useMCHLoss = True #only for Multiclass cases True: Multiclass-Hing Loss, False: Cross Entropy. 

#Bonsai uses one classier for Binary, thus this condition
if numClasses == 2:
    numClasses = 1

In [0]:
X = tf.placeholder("float32", [None, dataDimension])
Y = tf.placeholder("float32", [None, numClasses])

In [0]:
import datetime

def createTimeStampDir(dataDir):
    '''
    Creates a Directory with timestamp as it's name
    '''
    if os.path.isdir(dataDir + '/TFBonsaiResults') is False:
        try:
            os.mkdir(dataDir + '/TFBonsaiResults')
        except OSError:
            print("Creation of the directory %s failed" %
                  dataDir + '/TFBonsaiResults')

    currDir = 'TFBonsaiResults/' + datetime.datetime.now().strftime("%H_%M_%S_%d_%m_%y")
    if os.path.isdir(dataDir + '/' + currDir) is False:
        try:
            os.mkdir(dataDir + '/' + currDir)
        except OSError:
            print("Creation of the directory %s failed" %
                  dataDir + '/' + currDir)
        else:
            return (dataDir + '/' + currDir)
    return None

In [0]:
def saveMeanStdSeeDot(mean, std, seeDotDir):
    '''
    Function to save Mean and Std vectors
    '''
    if os.path.isdir(seeDotDir) is False:
        try:
            os.mkdir(seeDotDir)
        except OSError:
            print("Creation of the directory %s failed" %
                  seeDotDir)
    np.savetxt(seeDotDir + '/Mean', mean, delimiter="\t")
    np.savetxt(seeDotDir + '/Std', std, delimiter="\t")

In [0]:
def dumpCommand(list, currDir):
    '''
    Dumps the current command to a file for further use
    '''
    commandFile = open(currDir + '/command.txt', 'w')
    command = "python"

    command = command + " " + ' '.join(list)
    commandFile.write(command)

    commandFile.flush()
    commandFile.close()

In [0]:
currDir = createTimeStampDir(dataDir)
dumpCommand(sys.argv, currDir)
saveMeanStdSeeDot(mean, std, currDir)

In [0]:
bonsaiObj = Bonsai(numClasses, dataDimension, projectionDimension, depth, sigma)

W0716 15:41:21.574870 140428267288448 deprecation_wrapper.py:119] From /content/drive/My Drive/Dataset/EdgeML/tf/edgeml/graph/bonsai.py:70: The name tf.random_normal is deprecated. Please use tf.random.normal instead.



In [0]:
bonsaiTrainer = BonsaiTrainer(bonsaiObj, regW, regT, regV, regZ, sparW, sparT, sparV, sparZ,
                              learningRate, X, Y, useMCHLoss, outFile)

In [0]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [0]:
bonsaiTrainer.train(batchSize, totalEpochs, sess,
                    Xtrain, Xtest, Ytrain, Ytest, dataDir, currDir)


Epoch Number: 0

******************** Dense Training Phase Started ********************


Classification Train Loss: 1.790632068055921
Training accuracy (Classification): 0.8287201264823696
Test accuracy 0.867157
MarginLoss + RegLoss: 0.2991062 + 0.3078182 = 0.6069244


Epoch Number: 1

Classification Train Loss: 0.44351658567918084
Training accuracy (Classification): 0.8890101759307152
Test accuracy 0.896052
MarginLoss + RegLoss: 0.24224678 + 0.11576569 = 0.35801247


Epoch Number: 2

Classification Train Loss: 0.3452580441283969
Training accuracy (Classification): 0.9004327319892107
Test accuracy 0.909227
MarginLoss + RegLoss: 0.22963957 + 0.094196156 = 0.32383573


Epoch Number: 3

Classification Train Loss: 0.3230228577039938
Training accuracy (Classification): 0.9041752780960725
Test accuracy 0.910318
MarginLoss + RegLoss: 0.22086951 + 0.08052658 = 0.3013961


Epoch Number: 4

Classification Train Loss: 0.29962147897587416
Training accuracy (Classification): 0.906514369017255
Tes

# Fast Cells

In [0]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.

import helpermethods
import tensorflow as tf
import numpy as np
import sys
import os

#Provide the GPU number to be used
os.environ['CUDA_VISIBLE_DEVICES'] =''

#FastRNN and FastGRNN imports
from edgeml.trainer.fastTrainer import FastTrainer
from edgeml.graph.rnn import FastGRNNCell
from edgeml.graph.rnn import FastRNNCell
from edgeml.graph.rnn import UGRNNLRCell
from edgeml.graph.rnn import GRULRCell
from edgeml.graph.rnn import LSTMLRCell

# Fixing seeds for reproducibility
tf.set_random_seed(42)
np.random.seed(42)

In [0]:
def preProcessData(dataDir):
    '''
    Function to pre-process input data
    Expects a .npy file of form [lbl feats] for each datapoint,
    feats is timesteps*inputDims, flattened across timestep dimension.
    So input of 1st timestep followed by second and so on.
    Outputs train and test set datapoints
    dataDimension, numClasses are inferred directly
    '''
    
    train = np.load(os.path.join(dataDir, 'train_all_classes.npy'))
    test = np.load(os.path.join(dataDir, 'test_all_classes.npy'))
    
    '''
    train = np.load(os.path.join(dataDir, 'train.npy'))
    test = np.load(os.path.join(dataDir, 'test.npy'))
    '''
    dataDimension = int(train.shape[1]) - 1

    Xtrain = train[:, 1:dataDimension + 1]
    Ytrain_ = train[:, 0]
    numClasses = max(Ytrain_) - min(Ytrain_) + 1

    Xtest = test[:, 1:dataDimension + 1]
    Ytest_ = test[:, 0]

    numClasses = int(max(numClasses, max(Ytest_) - min(Ytest_) + 1))

    # Mean Var Normalisation
    mean = np.mean(Xtrain, 0)
    std = np.std(Xtrain, 0)
    std[std[:] < 0.000001] = 1
    Xtrain = (Xtrain - mean) / std

    Xtest = (Xtest - mean) / std
    # End Mean Var normalisation

    lab = Ytrain_.astype('uint8')
    lab = np.array(lab) - min(lab)

    lab_ = np.zeros((Xtrain.shape[0], numClasses))
    lab_[np.arange(Xtrain.shape[0]), lab] = 1
    Ytrain = lab_

    lab = Ytest_.astype('uint8')
    lab = np.array(lab) - min(lab)

    lab_ = np.zeros((Xtest.shape[0], numClasses))
    lab_[np.arange(Xtest.shape[0]), lab] = 1
    Ytest = lab_

    return dataDimension, numClasses, Xtrain, Ytrain, Xtest, Ytest, mean, std

In [0]:
#Loading and Pre-processing dataset for FastCells
#dataDir = "usps10"
dataDir = "sampled_dataset/"
#dataDir = "bot_iot_dataset/"

#(dataDimension, numClasses, Xtrain, Ytrain, Xtest, Ytest, mean, std) = helpermethods.preProcessData(dataDir)
(dataDimension, numClasses, Xtrain, Ytrain, Xtest, Ytest, mean, std) = preProcessData(dataDir)
print("Feature Dimension: ", dataDimension)
print("Num classes: ", numClasses)

Feature Dimension:  115
Num classes:  11


In [0]:
#cell = "FastGRNN" # Choose between FastGRNN, FastRNN, UGRNN, GRU and LSTM
#cell = "FastRNN"
#cell = "GRU"
#cell = "UGRNN"
cell = "LSTM"

inputDims = 23 #29 58 for sampled_dataset


#inputDims = 16 #features taken in by RNN in one timestep
hiddenDims = 32 #hidden state of RNN

totalEpochs = 20
batchSize = 100

learningRate = 0.01
decayStep = 200
decayRate = 0.1

outFile = None #provide your file, if you need all the logging info in a file

#low-rank parameterisation for weight matrices. None => Full Rank
wRank = None 
uRank = None 

#Sparsity of the weight matrices. x => 100*x % are non-zeros
sW = 1.0 
sU = 1.0

#Non-linearities for the RNN architecture. Can choose from "tanh, sigmoid, relu, quantTanh, quantSigm"
update_non_linearity = "tanh"
gate_non_linearity = "sigmoid"

assert dataDimension % inputDims == 0, "Infeasible per step input, Timesteps have to be integer"

In [0]:
X = tf.placeholder("float", [None, int(dataDimension / inputDims), inputDims])
Y = tf.placeholder("float", [None, numClasses])

In [0]:
import datetime

def createTimeStampDir(dataDir, cell):
    '''
    Creates a Directory with timestamp as it's name
    '''
    if os.path.isdir(os.path.join(dataDir, str(cell) + 'Results')) is False:
        try:
            os.mkdir(os.path.join(dataDir, str(cell) + 'Results'))
        except OSError:
            print("Creation of the directory %s failed" %
                  os.path.join(dataDir, str(cell) + 'Results'))

    currDir = os.path.join(str(cell) + 'Results',
        datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
    if os.path.isdir(os.path.join(dataDir, currDir)) is False:
        try:
            os.mkdir(os.path.join(dataDir, currDir))
        except OSError:
            print("Creation of the directory %s failed" %
                  os.path.join(dataDir, currDir))
        else:
            return (os.path.join(dataDir, currDir))
    return None

In [0]:
#currDir = helpermethods.createTimeStampDir(dataDir, cell)
currDir = createTimeStampDir(dataDir, cell)
helpermethods.dumpCommand(sys.argv, currDir)

In [0]:
#Create appropriate RNN cell object based on choice
if cell == "FastGRNN":
    FastCell = FastGRNNCell(hiddenDims, gate_non_linearity=gate_non_linearity,
                            update_non_linearity=update_non_linearity,
                            wRank=wRank, uRank=uRank)
elif cell == "FastRNN":
    FastCell = FastRNNCell(hiddenDims, update_non_linearity=update_non_linearity,
                           wRank=wRank, uRank=uRank)
elif cell == "UGRNN":
    FastCell = UGRNNLRCell(hiddenDims, update_non_linearity=update_non_linearity,
                           wRank=wRank, uRank=uRank)
elif cell == "GRU":
    FastCell = GRULRCell(hiddenDims, update_non_linearity=update_non_linearity,
                         wRank=wRank, uRank=uRank)
elif cell == "LSTM":
    FastCell = LSTMLRCell(hiddenDims, update_non_linearity=update_non_linearity,
                          wRank=wRank, uRank=uRank)
else:
    sys.exit('Exiting: No Such Cell as ' + cell)

In [0]:
FastCellTrainer = FastTrainer(FastCell, X, Y, sW=sW, sU=sU, learningRate=learningRate, outFile=outFile)

In [0]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
FastCellTrainer.train(batchSize, totalEpochs, sess, Xtrain, Xtest, Ytrain, Ytest, decayStep, decayRate, dataDir, currDir)


Epoch Number: 0

******************** Dense Training Phase Started ********************

Train Loss: 0.5612154866709854 Train Accuracy: 0.7053024998517952
Test Loss: 0.42163593 Test Accuracy: 0.7557108

Epoch Number: 1
Train Loss: 0.41326857456053145 Train Accuracy: 0.765506645043691
Test Loss: 0.44322008 Test Accuracy: 0.7649141

Epoch Number: 2
Train Loss: 0.32954314603950036 Train Accuracy: 0.8112036141482266
Test Loss: 0.30004606 Test Accuracy: 0.817963

Epoch Number: 3
Train Loss: 0.2842886553569274 Train Accuracy: 0.8322615618657584
Test Loss: 0.27061695 Test Accuracy: 0.83981496

Epoch Number: 4
Train Loss: 0.25718898788245037 Train Accuracy: 0.8437214233658531
Test Loss: 0.29274637 Test Accuracy: 0.8359449

Epoch Number: 5
Train Loss: 0.2545580920546946 Train Accuracy: 0.8458245606133432
Test Loss: 0.24047999 Test Accuracy: 0.84023976

Epoch Number: 6
Train Loss: 0.2506020051060301 Train Accuracy: 0.8509994695885013
Test Loss: 0.42968547 Test Accuracy: 0.7879932

Epoch Number: